# # Section 10

## NLP Fundamentals in TensorFLow

In [2]:
import tensorflow as tf
print("TensorFlow Version: ", tf.__version__)

TensorFlow Version:  2.9.0


In [4]:
from utils import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

### Load and visualize data

In [5]:
import pandas as pd
train_df = pd.read_csv("../datasets/nlp-getting-started/train.csv")
test_df = pd.read_csv("../datasets/nlp-getting-started/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df = train_df.sample(frac=1, random_state=42)
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)
len(train_sentences), len(val_sentences)

(6851, 762)

In [8]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [9]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [3]:
from tensorflow.keras.layers import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [9]:
text_vectorizer.adapt(train_sentences)

In [10]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [11]:
# Get the unique words in vocab:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_word = words_in_vocab[:5]
bottom_5_word = words_in_vocab[-5:]
print(len(words_in_vocab),)
print(top_5_word)
print(bottom_5_word)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Create an Embedding using an Embedding Layer

In [12]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

embedding

In [13]:
import random
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 To ouvindo sleeping with sirens awn      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00404893,  0.00221187,  0.01478701, ...,  0.03315074,
         -0.01167867, -0.02301277],
        [ 0.00365294, -0.00432271, -0.00480422, ..., -0.01499853,
          0.01695636, -0.00704441],
        [ 0.04772915,  0.01136645,  0.03447453, ..., -0.01719147,
         -0.0493711 , -0.04698119],
        ...,
        [ 0.02987665,  0.0323107 , -0.02486571, ..., -0.02976139,
         -0.0170548 , -0.03832744],
        [ 0.02987665,  0.0323107 , -0.02486571, ..., -0.02976139,
         -0.0170548 , -0.03832744],
        [ 0.02987665,  0.0323107 , -0.02486571, ..., -0.02976139,
         -0.0170548 , -0.03832744]]], dtype=float32)>

### Model 0: **Naive Bayes**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [15]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()),
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [16]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Accuracy score: {baseline_score*100:.2f}%")

Accuracy score: 79.27%


In [17]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [49]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

In [19]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: **Feed-forward neural network**

In [20]:
# Create a tensorboard callback (need to create a new one for each model)
from utils import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [21]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layers, sigmoid activation function for binary outputs

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")


In [22]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [23]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [24]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences, 
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230405-113518
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.6118 - accuracy: 0.6932 - val_loss: 0.5349 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4413 - accuracy: 0.8180 - val_loss: 0.4750 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3468 - accuracy: 0.8624 - val_loss: 0.4572 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2841 - accuracy: 0.8910 - val_loss: 0.4672 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2378 - accuracy: 0.9104 - val_loss: 0.4800 - val_accuracy: 0.7822


In [25]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4800 - accuracy: 0.7822


[0.4800453782081604, 0.7821522355079651]

In [26]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape, model_1_pred_probs[0]

24/24 [==============================] - 0s 2ms/step


((762, 1), array([0.3955172], dtype=float32))

In [27]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [28]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7856661242905698,
 'recall': 0.7821522309711286,
 'f1': 0.7795215466562155}

### Visualize learned embeddings

In [29]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [30]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
# Get the weight matrix of embedding layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[ 0.01867947,  0.04447923, -0.03638999, ..., -0.04239639,
        -0.00468699, -0.02810029],
       [-0.03501492, -0.03949334,  0.02028838, ...,  0.01179604,
         0.04798426,  0.00189335],
       [ 0.01011286,  0.00706809,  0.01564946, ..., -0.0244317 ,
         0.01667621, -0.00772734],
       ...,
       [-0.03829701, -0.02463633,  0.01208936, ..., -0.0377114 ,
         0.02033644,  0.03058371],
       [-0.0834004 ,  0.08386082, -0.01397063, ...,  0.00407728,
         0.00758088,  0.0777261 ],
       [-0.05040599,  0.10209696, -0.09694782, ..., -0.06395737,
         0.05332276,  0.10604609]], dtype=float32)

In [32]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


### Model 2: **LSTM model (RNN)**

In [43]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

# Create a lSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
# print(outputs.shape)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [44]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [46]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230405-115409
Epoch 1/5
215/215 [==============================] - 6s 13ms/step - loss: 0.5112 - accuracy: 0.7431 - val_loss: 0.4570 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3179 - accuracy: 0.8702 - val_loss: 0.5167 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2201 - accuracy: 0.9152 - val_loss: 0.5929 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1545 - accuracy: 0.9453 - val_loss: 0.6431 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1091 - accuracy: 0.9590 - val_loss: 0.8261 - val_accuracy: 0.7572


In [47]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[0]
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

24/24 [==============================] - 1s 8ms/step


{'accuracy': 75.7217847769029,
 'precision': 0.7577805667624024,
 'recall': 0.7572178477690289,
 'f1': 0.7554724830368379}

In [50]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 75.72, Difference: -3.54
Baseline precision: 0.81, New precision: 0.76, Difference: -0.05
Baseline recall: 0.79, New recall: 0.76, Difference: -0.04
Baseline f1: 0.79, New f1: 0.76, Difference: -0.03


### Model 3: **GRU model (RNN)**

### Model 4: **Bidirectional-LSTM model (RNN)**

### Model 5: **1D Convolutional Neural Network (CNN)**

### Model 6: **TensorFlow Hub Pretrained Feature Extraction**
(Using transfer learning for NLP)

### Model 7: **Model 6 with 10% of training data**